In [1]:
import clickhouse_connect
import os
import pandas as pd
from sqlalchemy import create_engine


In [2]:
clickhouse_host = os.getenv('CLICKHOUSE_HOST')
clickhouse_port = os.getenv('CLICKHOUSE_PORT')
clickhouse_username = os.getenv('CLICKHOUSE_USERNAME')
clickhouse_password = os.getenv('CLICKHOUSE_PASSWORD')
postgre_uri = os.getenv('POSTGRE_URI')
create_table_query = """CREATE TABLE bank_marketing
(   id int NOT NULL,
    age integer NOT NULL,
    job character varying(50),
    marital character varying(50),
    education character varying(50),
    "default" boolean,
    balance integer,
    housing boolean,
    loan boolean,
    contact character varying(50),
    day integer,
    month character varying(50),
    duration integer,
    campaign integer,
    pdays integer,
    previous integer,
    poutcome character varying(50),
    deposit boolean
)
ENGINE = MergeTree
PRIMARY KEY (id)"""
select_df_query = 'SELECT * FROM homebase.bank_marketing'
dropped_columns = ['created_at', 'updated_at']

In [3]:
client = clickhouse_connect.get_client(host=clickhouse_host, port=clickhouse_port, username=clickhouse_username, password=clickhouse_password)

In [4]:
client.command(create_table_query)

In [5]:
def pandas_execute_sql_postgre(uri, query):
    try:
        alchemyEngine = create_engine(uri, pool_recycle=3600)
        dbConnection = alchemyEngine.connect()
        dataFrame = pd.read_sql(query, dbConnection)
        dataFrame
        pd.set_option('display.expand_frame_repr', False)
        dbConnection.close()
        return dataFrame
    except Exception as e:
        print(e)
        return None
    finally:
        None

In [6]:
df = pandas_execute_sql_postgre(postgre_uri, select_df_query)

In [7]:
df = df.drop(columns=dropped_columns)

In [8]:
client.insert('bank_marketing', df)
